In [1]:
import digitalhub as dh

project = dh.get_or_create_project("test-taxonomy")

## Prepare base artifacts

In [2]:
import pandas as pd 

di1 = project.log_dataitem('taxonomy_nomi_di_comuni', kind="table", data=pd.read_csv("data-P1/nomi_di_comuni.txt", names=["name"]))
di2 = project.log_dataitem('taxonomy_stopwords', kind="table", data=pd.read_csv("data-P1/stopwords.txt", names=["word"]))
di3 = project.log_dataitem('taxonomy_termini_specifici', kind="table", data=pd.read_csv("data-P1/termini_specifici.txt", names=["termine"]))

In [3]:
import pandas as pd

tassonomia_df = pd.read_csv('data-P1/tassonomia_comuni.csv', sep=';')
di = project.log_dataitem(name="tassonomia_comuni", kind="table", data=tassonomia_df)

In [4]:
tassonomia_df = pd.read_csv('data-P1/tassonomia_aziende.csv', sep=';')
di = project.log_dataitem(name="tassonomia_aziende", kind="table", data=tassonomia_df)

## Compute azioni comuni

In [5]:
func = project.new_function(name="compute_azioni_comuni",
                            kind="python",
                            python_version="PYTHON3_10",
                            code_src="SuggerimentiComuni.py",
                            handler="taxonomy_suggestions",
                            requirements=["tqdm==4.66.6", "fuzzywuzzy==0.18.0", "python-Levenshtein==0.26.1", "simplemma==1.1.2", "unidecode==1.4.0"]
                           )

In [6]:
run = func.run(action="job",
              inputs={
                  'piani_comunali': project.get_dataitem('piani_comunali').key, 
                  'tassonomia': project.get_dataitem('tassonomia_comuni').key, 
                  'stopwords': project.get_dataitem('taxonomy_stopwords').key, 
                  'nomi': project.get_dataitem('taxonomy_nomi_di_comuni').key, 
                  'termini': project.get_dataitem('taxonomy_termini_specifici').key
              },
              wait=True)

2025-09-04 07:40:15,589 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:20,613 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:25,644 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:30,671 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:35,710 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:40,728 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:45,751 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:50,771 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:40:55,797 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:41:00,824 - INFO - Waiting for run 63757031b3154c3c982bae8a08ed0d29 to finish...
2025-09-04 07:41:05,871 - INFO - Waiting for run 6

## Service

In [7]:
comune_service = project.new_function(
        name="comune_taxonomy_service", 
        kind="python", 
        python_version="PYTHON3_10", 
        code_src="serve_comuni.py",     
        handler="serve",
        init_function="init"
    )

In [8]:
serve_run = comune_service.run(
    action="serve"
)

In [11]:
import requests

service_url = serve_run.refresh().status.service["url"]

res = requests.get(f"http://{service_url}/to_delete?threshold=3")
res.text

"[{'id': 87, 'frequency': 3}, {'id': 91, 'frequency': 3}, {'id': 81, 'frequency': 3}, {'id': 72, 'frequency': 2}, {'id': 84, 'frequency': 2}, {'id': 22, 'frequency': 1}, {'id': 15, 'frequency': 1}, {'id': 4, 'frequency': 1}, {'id': 88, 'frequency': 1}, {'id': 53, 'frequency': 1}, {'id': 80, 'frequency': 1}, {'id': 94, 'frequency': 1}, {'id': 122, 'frequency': 1}]"

In [12]:
res = requests.get(f"http://{service_url}/to_split?threshold=500")
res.text

"[{'id': 29, 'frequency': 1146}, {'id': 113, 'frequency': 989}, {'id': 68, 'frequency': 978}, {'id': 131, 'frequency': 905}, {'id': 28, 'frequency': 869}, {'id': 46, 'frequency': 810}, {'id': 54, 'frequency': 781}, {'id': 30, 'frequency': 695}, {'id': 196, 'frequency': 644}, {'id': 56, 'frequency': 622}, {'id': 26, 'frequency': 604}, {'id': 64, 'frequency': 589}, {'id': 62, 'frequency': 454}, {'id': 108, 'frequency': 443}, {'id': 21, 'frequency': 424}, {'id': 58, 'frequency': 408}, {'id': 3, 'frequency': 407}, {'id': 73, 'frequency': 401}, {'id': 11, 'frequency': 389}, {'id': 1, 'frequency': 383}, {'id': 114, 'frequency': 359}, {'id': 116, 'frequency': 356}, {'id': 110, 'frequency': 337}, {'id': 89, 'frequency': 335}, {'id': 9, 'frequency': 314}, {'id': 65, 'frequency': 311}, {'id': 90, 'frequency': 311}, {'id': 127, 'frequency': 306}, {'id': 51, 'frequency': 271}, {'id': 16, 'frequency': 268}, {'id': 66, 'frequency': 252}, {'id': 55, 'frequency': 248}, {'id': 23, 'frequency': 247}, {'

In [13]:
res = requests.get(f"http://{service_url}/to_merge")
res.text

'[{\'id1\': 68, \'id2\': 196, \'actions1\': \'progetto sceglilibro | progetto sceglilibro | progetto sceglilibro | visite e laboratori in biblioteca scuola primaria | laboratori per la scuola secondaria | rassegna teatrale per bambini | informazione e formazione per adulti nati per leggere | progetto di promozione alla lettura sceglilibro per elementare e media | spettacoli letture e laboratori nel periodo turistico | visite e laboratori in biblioteca per bambini della scuola primaria | laboratori per la scuola secondaria | rassegna teatrale per bambini | spettacoli letture laboratori nel periodo turistico | compartecipazione ai progetti sovra comunali | vietato ai maggiori | laboratori didattici ecomuseo | rassegna vietato ai maggiori | studio dei toponimi | spettacoli animazione bamini | spettacoli animazione per bambini | spettacoli e animazione per bambini | attivit culturali per bambini e ragazzi | mag scuole | organizzazione di eventi culturali | progetto pensi line | coccole di 